In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/daniilchernyshov/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


## Load Libraries and Data

In [ ]:
from itertools import combinations
from numba import njit, prange
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from catboost import CatBoostRegressor
from catboost import EShapCalcType, EFeaturesSelectionAlgorithm
from catboost import Pool
import shap

In [2]:
train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

features = [col for col in train.columns if col not in ['row_id', 'time_id', 'target']]
print(features)

['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap']


In [ ]:
train = train.dropna(subset=["target"])
train.reset_index(drop=True, inplace=True)

In [ ]:
X = train[features].copy(deep=True)
y = train['target'].copy(deep=True)

## Run Full Process or Do Submission

Below is a trigger to run the full feature selection and hyperparam tuning process or to simply make you make your final submission.

In [ ]:
submit = True

## Feature Engineering functions

In [ ]:
# Function to compute triplet imbalance in parallel using Numba
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    # Loop through all combinations of triplets
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]

        # Loop through rows of the DataFrame
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val

            # Prevent division by zero
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features


# Function to calculate triplet imbalance for given price data and a DataFrame
def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance using the Numba-optimized function
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    
    return features


def numba_imb_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)

    # Calculate triplet imbalance features using the Numba-optimized function
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    return df

In [1]:
def feature_engineering(X):

    _X = X.copy()

    # features taken from https://www.kaggle.com/code/zulqarnainali/explained-singel-model-optiver
    # market_urgency', 'seconds_in_bucket', 'liquidity_imbalance', 'imbalance_momentum', 'price_spread', 'matched_imbalance', 'bid_size', 'matched_size', 'spread_intensity', 'ask_size'
    _X["volume"] = _X.eval("ask_size + bid_size")
    _X["mid_price"] = _X.eval("(ask_price + bid_price) / 2")
    _X["liquidity_imbalance"] = _X.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    _X["matched_imbalance"] = _X.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    _X["size_imbalance"] = _X.eval("bid_size / ask_size")

    _X["imbalance_momentum"] = _X.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / _X['matched_size']
    _X["price_spread"] = _X["ask_price"] - _X["bid_price"]
    _X["spread_intensity"] = _X.groupby(['stock_id'])['price_spread'].diff()
    _X['price_pressure'] = _X['imbalance_size'] * (_X['ask_price'] - _X['bid_price'])
    _X['market_urgency'] = _X['price_spread'] * _X['liquidity_imbalance']
    _X['depth_pressure'] = (_X['ask_size'] - _X['bid_size']) * (_X['far_price'] - _X['near_price'])

    _X["dow"] = _X["date_id"] % 5  # Day of the week
    _X["seconds"] = _X["seconds_in_bucket"] % 60  # Seconds
    _X["minute"] = _X["seconds_in_bucket"] // 60  # Minutes

    # # Calculate shifted and return features for specific columns
    for col in ['matched_size', 'imbalance_size', 'reference_price']:
        for window in [1, 2, 3, 10]:
            _X[f"{col}_shift_{window}"] = _X.groupby('stock_id')[col].shift(window)
            _X[f"{col}_ret_{window}"] = _X.groupby('stock_id')[col].pct_change(window)

    # # Calculate diff features for specific columns
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size']:
        for window in [1, 2, 3, 10]:
            _X[f"{col}_diff_{window}"] = _X.groupby("stock_id")[col].diff(window)

    # Create features for pairwise price imbalances
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    for c in combinations(prices, 2):
        _X[f"{c[0]}_{c[1]}_imb"] = _X.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    # Generate imbalance features
    _X = numba_imb_features(_X)

    _X = _X.replace([np.inf, -np.inf], 0)
    _X.drop(columns=['date_id'], inplace=True)

    return _X

## Train/test Split

In [ ]:
if not submit:
    
    number_of_days = len(X.date_id.unique()) # There are 481 trading days, we'll take the last n as testing set
    testing_days = 10
    training_days = number_of_days - testing_days
    training_days, testing_days

    # mask to grab the days for training and testing
    training_mask = X.date_id <= training_days
    testing_mask = X.date_id > training_days

    # subset and make training and validation sets
    X_train, X_val, y_train, y_val = X[training_mask], X[testing_mask], y[training_mask], y[testing_mask] #train_test_split(X, y, test_size=0.20, random_state=0, shuffle=True, stratify=X['stock_id']) # random_state=8
    X_train.shape, X_val.shape, y_train.shape, y_val.shape
    
    # apply feature engineering
    X_train = feature_engineering(X_train)
    X_val = feature_engineering(X_val)

    # Convert all numerics to float32 to reduce memory footprint
    X_train = X_train.astype(np.float32)
    X_val = X_val.astype(np.float32)
    
    X_train.shape, X_val.shape

## Feature Selection

In [ ]:
if not submit: 
    %%time
    def select_features(model, algorithm, num_features, steps):

        print('Algorithm:', algorithm)

        summary = model.select_features(
            train_pool,
            eval_set=val_pool,
            features_for_select=list(range(train_pool.num_col())),
            num_features_to_select=num_features,
            steps=steps,
            algorithm=algorithm,
            shap_calc_type=EShapCalcType.Regular,
            train_final_model=True,
            logging_level='Silent',
            plot=True
        )

        print('Selected features:\n', summary['selected_features_names'])
        print('Eliminated features:\n', summary['eliminated_features_names'])

        return summary

    train_pool = Pool(data=X_train, label=y_train)
    val_pool = Pool(data=X_val, label=y_val)

    # baseline parameters
    params = dict(loss_function='MAE',
                  eval_metric = 'MAE',
                  metric_period=100,
                  task_type='GPU',
                  od_type='Iter',
                  od_wait=50,
                  bootstrap_type='Bernoulli',
                  )

    # run the feature selection algorithm
    model = CatBoostRegressor(**params)
    summary = select_features(model=model, algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues, num_features=50, steps=3)

In [ ]:
# summary = {}
# feature selection process turned off for submission
# summary['selected_features_names'] = ['seconds_in_bucket', 'imbalance_buy_sell_flag', 'bid_size', 'ask_size', 'volume', 
#                                       'liquidity_imbalance', 'size_imbalance', 'imbalance_momentum', 'price_spread', 'spread_intensity', 
#                                       'market_urgency', 'minute', 'matched_size_ret_1', 'matched_size_shift_10', 'imbalance_size_ret_1', 
#                                       'imbalance_size_ret_2', 'imbalance_size_ret_3', 'imbalance_size_ret_10', 'ask_price_diff_2', 
#                                       'ask_price_diff_3', 'bid_price_diff_2', 'bid_price_diff_3', 'reference_price_near_price_imb', 
#                                       'reference_price_ask_price_imb', 'reference_price_bid_price_imb', 'reference_price_wap_imb', 
#                                       'far_price_ask_price_imb', 'far_price_bid_price_imb', 'far_price_wap_imb', 'near_price_ask_price_imb', 
#                                       'near_price_bid_price_imb', 'near_price_wap_imb', 'ask_price_bid_price_imb', 'ask_price_wap_imb', 
#                                       'bid_price_wap_imb', 'all_prices_mean', 'all_sizes_mean', 'all_prices_std', 'all_sizes_std', 
#                                       'all_prices_skew', 'all_sizes_skew', 'all_prices_kurt', 'all_sizes_kurt', 'ask_price_bid_price_wap_imb2', 
#                                       'ask_price_bid_price_reference_price_imb2', 'ask_price_wap_reference_price_imb2', 'bid_price_wap_reference_price_imb2', 
#                                       'matched_size_bid_size_ask_size_imb2', 'matched_size_bid_size_imbalance_size_imb2', 'matched_size_ask_size_imbalance_size_imb2']

In [ ]:
if not submit:
    
    %%time

    # https://medium.com/analytics-vidhya/catboost-101-fb2fdc3398f3

    # Retrain the model with the recommended number of iterations and the reduced feature set
    final_params = params.copy()
    final_params["iterations"] = model.best_iteration_
    print(final_params)

    # fit the final model
    final_model = CatBoostRegressor(**final_params)
    final_model.fit(X_train[summary['selected_features_names']], y_train)

    # predict with the final model
    y_val_pred = final_model.predict(X_val[summary['selected_features_names']])

    # get the validation set score
    y_val_mae = mean_absolute_error(y_val, y_val_pred)
    print(f"MAE on validation set: {y_val_mae}")

In [ ]:
if not submit:
    
    %%time
    
    # Retrain the model with the recommended number of iterations and the reduced feature set
    exper_params = params.copy()
    exper_params["iterations"] = model.best_iteration_
    print(exper_params)

    # fit the experimental model
    exper_model = CatBoostRegressor(**final_params)
    exper_model.fit(X_train[summary['selected_features_names']], y_train)

    # predict with the final model
    y_val_pred = exper_model.predict(X_val[summary['selected_features_names']])

    # get the validation set score
    y_val_mae = mean_absolute_error(y_val, y_val_pred)
    print(f"MAE on validation set: {y_val_mae}")
    
    # get the feature importances and plot them
    feat_importances = exper_model.get_feature_importance(prettified=True)
    plt.figure(figsize=(12, 10))
    sns.barplot(x="Importances", y="Feature Id", data=feat_importances[0:10])
    plt.title('Top features:')

## K-fold CV with selected features to find best hyperparams

In [ ]:
if not submit:
    # 10-fold CV to find best hyperparameters
    splitter = KFold(n_splits=10, shuffle=True)
    splits = []
    for i, (train_index, test_index) in enumerate(splitter.split(X_train, y_train)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}, length={len(train_index)}")
        print(f"  Test:  index={test_index},  length={len(test_index)}")
        splits.append((train_index, test_index))

In [ ]:
if not submit:
    
    %%time
    
    # baseline model
    params = dict(loss_function='MAE',
                  eval_metric = 'MAE',
                  metric_period=100,
                  task_type='GPU',
                  od_type='Iter',
                  od_wait=50,
                  bootstrap_type='Bernoulli'
                  )
    model = CatBoostRegressor(**params)
    
    # search these params
    param_distributions = {"iterations": [int(x) for x in np.linspace(start=200, stop=1000, num=15)],
                           "depth": list(range(1, 16)),
                           "subsample": loguniform(0.1, 1),
                           "random_strength": np.linspace(start=1, stop=25, num=15),
                           "learning_rate": loguniform(0.01, 1),
                           "l2_leaf_reg": [int(x) for x in np.linspace(start=1, stop=40, num=20)],
                           "score_function": ['L2', 'Cosine', 'NewtonL2', 'NewtonCosine']
                          }
    param_distributions

In [ ]:
if not submit:
    
    %%time

    # HalvingRandomSearchCV
    search_cv = HalvingRandomSearchCV(
        model,
        param_distributions=param_distributions,
        scoring="neg_mean_absolute_error",
        cv = splits,
        n_candidates=200,
        min_resources=100000,
        max_resources=1000000,
        random_state=7,
        verbose=1
    )

    search_cv.fit(X_train, y_train)
    y_pred = search_cv.best_estimator_.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred) # 5.838548462599918 with 30 days validation data
    print('Best search_cv score:' , search_cv.best_score_)
    print('Validation set MAE: ', mae)
    
    columns = [f"param_{name}" for name in param_distributions.keys()]
    columns += ["mean_test_score", "std_test_score"]
    cv_results = pd.DataFrame(search_cv.cv_results_)
    cv_results.sort_values(by="mean_test_score", ascending=False, inplace=True)
    cv_results[columns].head()

In [ ]:
if not submit: 
    search_cv.best_estimator_.get_params()
# 'iterations': 771,
#          'learning_rate': 0.6153054327622673,
#          'depth': 13,
#          'l2_leaf_reg': 9,
#          'loss_function': 'MAE',
#          'od_wait': 50,
#          'od_type': 'Iter',
#          'metric_period': 100,
#          'random_strength': 4.428571428571429,
#          'eval_metric': 'MAE',
#          'task_type': 'GPU',
#          'bootstrap_type': 'Bernoulli',
#          'subsample': 0.9795657524290455,
#          'score_function': 'Cosine'

## Train Final Model

In [ ]:
# apply feature engineering
X = feature_engineering(X)

# Convert all numerics to float32 to reduce memory footprint
X = X.astype(np.float32)
X.shape

In [ ]:
# selected_features = ['seconds_in_bucket', 'imbalance_buy_sell_flag', 'bid_size', 'ask_size', 'volume', 'liquidity_imbalance', 
#                      'size_imbalance', 'imbalance_momentum', 'price_spread', 'spread_intensity', 'market_urgency', 'minute', 
#                      'matched_size_ret_1', 'matched_size_shift_10', 'imbalance_size_ret_1', 'imbalance_size_ret_2', 
#                      'imbalance_size_ret_3', 'imbalance_size_ret_10', 'ask_price_diff_2', 'ask_price_diff_3', 
#                      'bid_price_diff_2', 'bid_price_diff_3', 'reference_price_near_price_imb', 'reference_price_ask_price_imb', 
#                      'reference_price_bid_price_imb', 'reference_price_wap_imb', 'far_price_ask_price_imb', 'far_price_bid_price_imb', 
#                      'far_price_wap_imb', 'near_price_ask_price_imb', 'near_price_bid_price_imb', 'near_price_wap_imb', 'ask_price_bid_price_imb', 
#                      'ask_price_wap_imb', 'bid_price_wap_imb', 'all_prices_mean', 'all_sizes_mean', 'all_prices_std', 'all_sizes_std', 
#                      'all_prices_skew', 'all_sizes_skew', 'all_prices_kurt', 'all_sizes_kurt', 'ask_price_bid_price_wap_imb2', 
#                      'ask_price_bid_price_reference_price_imb2', 'ask_price_wap_reference_price_imb2', 'bid_price_wap_reference_price_imb2', 
#                      'matched_size_bid_size_ask_size_imb2', 'matched_size_bid_size_imbalance_size_imb2', 'matched_size_ask_size_imbalance_size_imb2']

In [ ]:
#X = X[selected_features]
X.shape

In [ ]:
%%time

params = {'iterations': 771,
         'learning_rate': 0.6153054327622673,
         'depth': 13,
         'l2_leaf_reg': 9,
         'loss_function': 'MAE',
         'od_wait': 50,
         'od_type': 'Iter',
         'metric_period': 100,
         'random_strength': 4.428571428571429,
         'eval_metric': 'MAE',
         'task_type': 'GPU',
         'bootstrap_type': 'Bernoulli',
         'subsample': 0.9795657524290455,
         'score_function': 'Cosine'}

final_model = CatBoostRegressor(**params)
final_model.fit(X, y)

In [ ]:
feat_importances = final_model.get_feature_importance(prettified=True)

plt.figure(figsize=(12, 10))
sns.barplot(x="Importances", y="Feature Id", data=feat_importances[0:10])
plt.title('Top features:');

## Make Submission

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices) / np.sum(std_error)
    out = prices - std_error * step
    return out

In [ ]:
counter = 0
y_min, y_max = -64, 64
predictions = []
cache = pd.DataFrame()
for (test, revealed_targets, sample_prediction) in iter_test:
    
    cache = pd.concat([cache, test], ignore_index=True, axis=0)
    if counter > 0:
        cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
    X_test = cache[features]
    X_test = feature_engineering(X_test)[-len(test):]
    X_test = X_test.astype(np.float32)
    preds = final_model.predict(X_test)
    preds = zero_sum(preds, test['bid_size'] + test['ask_size'])
    preds = np.clip(preds, y_min, y_max)
    sample_prediction['target'] = preds
    env.predict(sample_prediction)
    counter += 1

In [ ]:
sample_prediction.hist(column='target', bins=100, range=[-10,10])

In [ ]:
sample_prediction.to_csv('preds.csv')